In [27]:
import numpy as np
import scipy.io
mat = scipy.io.loadmat("/media/allen/mass/shanshanzhang-citypersons-dec0e5b2f30a/annotations/anno_val.mat")
# data = np.array(data)

In [83]:
dataset = {
    "info": {},
    "licenses": [],
    "images": [],
    "annotations": [],
    "categories": []
}
dataset['categories'].append({
    'id': 1,
    'name': "person",
    'supercategory': "people",
    'keypoints': [],
    'skeleton': []
})

# city_name; im_name; bbs
sub_index = 0
for idx, anns in enumerate(mat['anno_val_aligned'][0]):
    city, im_name, bbs = anns[0][0]
      
    dataset['images'].append({
        'coco_url': '',
        'date_captured': '',
        'file_name': str(city) + "/" + str(im_name[0]),
        'flickr_url': '',
        'id': idx,
        'license': 0,
        'width': 2048,
        'height': 1024
    })
    for bb in bbs:
        sub_index = sub_index + 1
        class_label, x1, y1, w, h, instance_id, x1_vis, y1_vis, w_vis, h_vis = bb 
        if class_label == 0:
            continue
        dataset['annotations'].append({
            'area': int(w*h),
            'bbox': [int(x1),int(y1),int(w),int(h)],
            'category_id': 1,
            'id': sub_index,
            'image_id': idx,
            'iscrowd': 0,
            'segmentation': [],
        })

/home/allen/.pyenv/versions/3.6.8/envs/py368/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: overflow encountered in ushort_scalars
/home/allen/.pyenv/versions/3.6.8/envs/py368/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: overflow encountered in short_scalars


In [79]:
type(x1)

numpy.uint16

In [51]:
import pycocotools.coco as coco
import json

In [84]:
with open('/media/allen/mass/DB/cityperson/original/instances_val2017.json', 'w') as f:
    json.dump(dataset, f)

In [85]:
co = coco.COCO('/media/allen/mass/DB/cityperson/original/instances_val2017.json')

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [89]:
image_ids = co.getImgIds()
img_id = image_ids[0]
idxs = co.getAnnIds(imgIds=[img_id])
fname = co.loadImgs(ids=[img_id])[0]['file_name']

In [90]:
idxs

[1, 2, 3, 4]